In [32]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from tensorflow.keras import Sequential, layers, losses, optimizers, models
import tensorflow as tf
import os
from csv import reader
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [33]:
seed_num = 128
import random as python_random
np.random.seed(seed_num)
python_random.seed(seed_num)
tf.random.set_seed(seed_num)
from pickle import load
load_minmax_scaler = load(open('./minmax_scaler.pkl', 'rb'))

In [34]:
checkpoint_path = './temp2/checkpoint.ckpt'
optimizer = optimizers.Adam()
model = Sequential()
model.add(tf.keras.layers.Dense(38, input_dim =38,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(9,activation = 'relu'))
model.add(tf.keras.layers.Dense(3,  activation='softmax'))
model.compile(
    loss=sparse_categorical_crossentropy, 
    optimizer=optimizer, 
    metrics=['acc'])
model.summary()
model.load_weights(checkpoint_path)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 38)                1482      
                                                                 
 dense_6 (Dense)             (None, 19)                741       
                                                                 
 dense_7 (Dense)             (None, 19)                380       
                                                                 
 dense_8 (Dense)             (None, 9)                 180       
                                                                 
 dense_9 (Dense)             (None, 3)                 30        
                                                                 
Total params: 2,813
Trainable params: 2,813
Non-trainable params: 0
_________________________________________________________________


In [35]:
excel_data=pd.read_csv('ResultWeight_MainDimension.csv',header=None)

In [36]:
num_data = excel_data.to_numpy()
inx = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,25,26,27,28,29,30,31,32,33,34,35,36]
data1 = num_data[:,inx]

In [37]:
폭 = np.average(num_data[:,37:41],axis=1)
높이 = np.average(num_data[:,41:45],axis=1)
면적 = np.average(num_data[:,49:53],axis=1)
둘레 = np.average(num_data[:,53:57],axis=1)

면적둘레 = 면적/둘레
면적폭 = 면적/폭
면적높이 = 면적/높이
폭=np.reshape(폭,(num_data.shape[0],1))
높이=np.reshape(높이,(num_data.shape[0],1))
면적=np.reshape(면적,(num_data.shape[0],1))
둘레=np.reshape(둘레,(num_data.shape[0],1))
면적둘레=np.reshape(면적둘레,(num_data.shape[0],1))
면적폭=np.reshape(면적폭,(num_data.shape[0],1))
면적높이=np.reshape(면적높이,(num_data.shape[0],1))

In [38]:
data2 = np.concatenate((data1,폭,높이,면적,둘레,면적둘레,면적폭,면적높이,num_data[:,57:59]),axis=1)
data2[0]

array([980.01, 428.64, 347.59, 598.48, 160031.48, 1587.94, 355.86, 373.59,
       619.81, 632.45, 175869.68, 189565.66, 1613.3, 1655.67, 388.17,
       626.65, 202825.1, 1672.26, 381.94, 386.67, 604.5, 623.24,
       186301.44, 202817.26, 1622.83, 1674.97, 423.54, 373.74, 590.59,
       183721.06, 1644.1, 296.305, 463.41999999999996, 143775.5325,
       1238.425, 116.0954700526879, 485.2281686100471, 310.2488725130551,
       279, 2], dtype=object)

In [39]:
num_data2 = data2

돼지파일명 = num_data[0:,0]
치수 = num_data2[0:,0:-2]
체형 = num_data2[0:,-1].astype('float32')
print(f'치수_shape = {치수.shape}\n체형_shape = {체형.shape}') #35개의 선별 치수 + 둘레, 폭, 높이당 면적

치수_shape = (100, 38)
체형_shape = (100,)


In [40]:
result = model.predict(load_minmax_scaler.transform(치수))
Results = np.argmax(result,axis =1)
Results = Results +1

4/4 [==============================] - 0s 4ms/step


In [41]:
cnt =0

for i in range(100):
    if(Results[i] == 체형[i]):
        cnt = cnt +1
print(f"acc is {cnt/100*100}")

acc is 47.0


In [42]:
Results

array([2, 2, 3, 3, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2], dtype=int64)

In [13]:
excel_data=pd.read_excel('돼지치수_70마리_1022개.xlsx', sheet_name='체형데이터_1022', header=1)

In [14]:
num_data = excel_data.to_numpy()
Corr = num_data[0,1:]
돼지파일명 = num_data[1:,0]
치수 = num_data[1:,1:-1]
체형 = num_data[1:,-1].astype('float32')
print(f'치수_shape = {치수.shape}\n체형_shape = {체형.shape}') #35개의 선별 치수 + 둘레, 폭, 높이당 면적
치수_train, 체형_train, 치수_test, 체형_test = 치수[:696], 체형[:696], 치수[696:], 체형[696:]

치수_shape = (1022, 38)
체형_shape = (1022,)


In [15]:
from pickle import load
load_minmax_scaler = load(open('./minmax_scaler.pkl', 'rb'))

In [16]:
치수_scaled = load_minmax_scaler.transform(치수_train)

In [17]:
checkpoint_path = './temp2/checkpoint.ckpt'
optimizer = optimizers.Adam()
model = Sequential()
model_check_point = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)
model.add(tf.keras.layers.Dense(38, input_dim =38,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(9,activation = 'relu'))
model.add(tf.keras.layers.Dense(4,  activation='softmax'))
model.compile(
    loss=sparse_categorical_crossentropy, 
    optimizer=optimizer, 
    metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 38)                1482      
                                                                 
 dense_1 (Dense)             (None, 19)                741       
                                                                 
 dense_2 (Dense)             (None, 19)                380       
                                                                 
 dense_3 (Dense)             (None, 9)                 180       
                                                                 
 dense_4 (Dense)             (None, 4)                 40        
                                                                 
Total params: 2,823
Trainable params: 2,823
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.load_weights(checkpoint_path)

In [19]:
X_Unlabeled_test = load_minmax_scaler.transform(치수_test)

In [20]:
Y_Unlabeled_pred = model.predict(X_Unlabeled_test)
Y_Unlabeled_pred = np.argmax(Y_Unlabeled_pred, axis=1)

11/11 [==============================] - 0s 1ms/step


In [21]:
체형_test = Y_Unlabeled_pred

In [22]:
X_merge = np.concatenate((치수_train, 치수_test), axis = 0)
y_merge = np.concatenate((체형_train, 체형_test))

In [23]:
Y_Unlabeled_pred

array([3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 3, 3,
       3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3,

In [24]:
y_merge = np.reshape(y_merge,(1022,1))

In [25]:
#y_merge = np.reshape(y_merge, (X_merge.shape[0],1))

In [26]:
data_merge = np.concatenate((X_merge,y_merge), axis = 1)
data_merge = pd.DataFrame(data_merge)

In [27]:
data_merge.shape

(1022, 39)

In [28]:
data_DataFrame = pd.DataFrame(data_merge)

In [29]:
data_DataFrame.to_excel('Data_result.xlsx')